In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nottollfree","key":"76e74913b237c377a9042f8092186788"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d shyambhu/hands-and-palm-images-dataset


Dataset URL: https://www.kaggle.com/datasets/shyambhu/hands-and-palm-images-dataset
License(s): DbCL-1.0


In [ ]:
!unzip hands-and-palm-images-dataset.zip -d 11k_hands


Streaming output truncated to the last 5000 lines.
  inflating: 11k_hands/Hands/Hands/Hand_0006508.jpg  
  inflating: 11k_hands/Hands/Hands/Hand_0006509.jpg  
  inflating: 11k_hands/Hands/Hands/Hand_0006510.jpg  
  inflating: 11k_hands/Hands/Hands/Hand_0006511.jpg  
  inflating: 11k_hands/Hands/Hands/Hand_0006512.jpg  
  inflating: 11k_hands/Hands/Hands/Hand_0006513.jpg  
  inflating: 11k_hands/Hands/Hands/Hand_0006514.jpg  
  inflating: 11k_hands/Hands/Hands/Hand_0006515.jpg  
  inflating: 11k_hands/Hands/Hands/Hand_0006516.jpg  
  inflating: 11k_hands/Hands/Hands/Hand_0006517.jpg  
  inflating: 11k_hands/Hands/Hands/Hand_0006518.jpg  
  inflating: 11k_hands/Hands/Hands/Hand_0006519.jpg  
  inflating: 11k_hands/Hands/Hands/Hand_0006520.jpg  
  inflating: 11k_hands/Hands/Hands/Hand_0006521.jpg  
  inflating: 11k_hands/Hands/Hands/Hand_0006522.jpg  
  inflating: 11k_hands/Hands/Hands/Hand_0006523.jpg  
  inflating: 11k_hands/Hands/Hands/Hand_0006524.jpg  
  inflating: 11k_hands/Hands/Ha

In [ ]:
!ls /content/11k_hands


HandInfo.csv  Hands


# New Section

In [ ]:
!ls -R /content/11k_hands | head -30


/content/11k_hands:
HandInfo.csv
Hands

/content/11k_hands/Hands:
Hands

/content/11k_hands/Hands/Hands:
Hand_0000002.jpg
Hand_0000003.jpg
Hand_0000004.jpg
Hand_0000005.jpg
Hand_0000006.jpg
Hand_0000007.jpg
Hand_0000008.jpg
Hand_0000009.jpg
Hand_0000010.jpg
Hand_0000011.jpg
Hand_0000012.jpg
Hand_0000013.jpg
Hand_0000014.jpg
Hand_0000015.jpg
Hand_0000016.jpg
Hand_0000020.jpg
Hand_0000021.jpg
Hand_0000022.jpg
Hand_0000023.jpg
Hand_0000024.jpg
Hand_0000025.jpg
Hand_0000026.jpg


In [ ]:
import os
import cv2
import numpy as np
import random
from tqdm import tqdm

input_dir = '/content/11k_hands/Hands/Hands'  # ✅ correct path to your images
output_base = '/content/hand_dataset_labeled'

clean_dir = os.path.join(output_base, 'clean')
dirty_dir = os.path.join(output_base, 'dirty')

os.makedirs(clean_dir, exist_ok=True)
os.makedirs(dirty_dir, exist_ok=True)

all_images = [f for f in os.listdir(input_dir) if f.endswith(('.jpg', '.png'))]
random.shuffle(all_images)

# Split: 70% clean, 30% dirty
num_dirty = int(0.3 * len(all_images))
dirty_imgs = set(all_images[:num_dirty])

def add_dirt_spots(img, num_spots=5):
    h, w = img.shape[:2]
    for _ in range(num_spots):
        center = (random.randint(0, w), random.randint(0, h))
        radius = random.randint(5, 25)
        color = (random.randint(30, 60), random.randint(30, 60), random.randint(30, 60))  # dark brown
        cv2.circle(img, center, radius, color, -1)
    return img

for img_name in tqdm(all_images):
    img_path = os.path.join(input_dir, img_name)
    img = cv2.imread(img_path)
    if img is None:
        continue

    if img_name in dirty_imgs:
        img = add_dirt_spots(img)
        save_path = os.path.join(dirty_dir, img_name)
    else:
        save_path = os.path.join(clean_dir, img_name)

    cv2.imwrite(save_path, img)


100%|██████████| 11076/11076 [02:23<00:00, 77.34it/s]


In [ ]:
!ls /content/hand_dataset_labeled/clean | wc -l
!ls /content/hand_dataset_labeled/dirty | wc -l


7754
3322


In [ ]:
!pip install tensorflow

In [ ]:
print("Clean folder contains:", len(os.listdir('/content/hand_dataset_labeled/clean')), "images")
print("Dirty folder contains:", len(os.listdir('/content/hand_dataset_labeled/dirty')), "images")

Clean folder contains: 7754 images
Dirty folder contains: 3322 images


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import os

# Paths
train_dir = '/content/hand_dataset_labeled'

# Image size and batch
img_size = (160, 160)
batch_size = 32

# Data preprocessing
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_data = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# MobileNetV2 base
base_model = MobileNetV2(input_shape=img_size + (3,), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train for 20 epochs
model.fit(train_data, epochs=20, validation_data=val_data)


Found 8862 images belonging to 2 classes.
Found 2214 images belonging to 2 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 110s 359ms/step - accuracy: 0.8557 - loss: 0.3539 - val_accuracy: 0.9625 - val_loss: 0.1186
Epoch 2/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 87s 316ms/step - accuracy: 0.9825 - loss: 0.0702 - val_accuracy: 0.9720 - val_loss: 0.0809
Epoch 3/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 90s 325ms/step - accuracy: 0.9906 - loss: 0.0422 - val_accuracy: 0.9747 - val_loss: 0.0722
Epoch 4/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 140s 317ms/step - accuracy: 0.9951 - loss: 0.0285 - val_accuracy: 0.9774 - val_loss: 0.0681
Epoch 5/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 86s 310ms/step - accuracy: 0.9957 - loss: 0.0232 - val_accuracy: 0.9806 - val_loss: 0.0633
Epoch 6/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 88s 318ms/step - accuracy: 0.9960 - loss: 0.0210 - val_accuracy: 0.9783 - val_loss: 0.0680
Epoch 7/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 90s 327ms/step - accuracy: 0.9969 - loss: 0.0178 - val_accuracy: 0.9788 - val_loss: 0.0691
Epoch 8/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 87s 314ms/step - accuracy: 0.9980 - loss:

In [ ]:
# Save the model
model.save('hand_cleanliness_20ep.keras')

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # optimize for ESP32
tflite_model = converter.convert()

with open('hand_cleanliness_20ep.tflite', 'wb') as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmpkqn3q9w4'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 160, 160, 3), dtype=tf.float32, name='keras_tensor_157')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  136797491831696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136797491831504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136797491827472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136797491834960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136797491838224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136797491836496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136797491835728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136797491837072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136797491835152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136797491835536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1367974918

In [ ]:
from google.colab import files
files.download('hand_cleanliness_20ep.keras')
files.download('hand_cleanliness_20ep.tflite')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    prediction = model.predict(img_array)[0][0]
    label = "Dirty" if prediction >= 0.5 else "Clean"
    print(f"Prediction: {label} ({prediction:.2f})")


In [ ]:
predict_image('/content/hand_dataset_labeled/dirty/Hand_0000010.jpg')


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
Prediction: Clean (0.33)


In [ ]:
predict_image('/content/hand_dataset_labeled/dirty/Hands/Hands/Hand_0000010.jpg')


FileNotFoundError: [Errno 2] No such file or directory: '/content/hand_dataset_labeled/dirty/Hands/Hands/Hand_0000010.jpg'

In [ ]:
!ls

11k_hands  hand_dataset_labeled  hands-and-palm-images-dataset.zip  sample_data


In [ ]:
!find /content/hand_dataset_labeled -name "Hand_0000010.jpg"


/content/hand_dataset_labeled/clean/Hand_0000010.jpg


In [ ]:
predict_image('/content/hand_dataset_labeled/clean/Hand_0000010.jpg')

FileNotFoundError: [Errno 2] No such file or directory: '/content/hand_dataset_labeled/clean/Hand_0000010.jpg'

In [ ]:
model.save('/content/hand_classifier_model.h5')


In [ ]:
model.save('my_new_model.keras')

In [ ]:
!ls

11k_hands	      hands-and-palm-images-dataset.zip  sample_data
hand_dataset_labeled  my_new_model.keras


In [ ]:
from google.colab import files
files.download('my_new_model.keras')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import tensorflow as tf

# Load your model
model = tf.keras.models.load_model('my_new_model.keras')

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Optimize (optional but highly recommended for ESP32)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert and save
tflite_model = converter.convert()

# Save to file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmpx396zcq4'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  134145164843664: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  134145164846544: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  134148137563152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134148137565648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134145956807312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134148137562768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134148137562384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134145956807696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134145956808656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134145956809616: TensorSpec(shape=(), dtype=tf.resource, name=Non

In [ ]:
from google.colab import files
files.download('model.tflite')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>